In [51]:
import clingo

from starlingo.Atom import Atom
from starlingo.Literal import BasicLiteral
from starlingo.Program import from_string, Program, evaluate_forwards
from starlingo.Rule import NormalRule, Constraint
from starlingo.Symbol import Function, IntegerConstant, Term

In [52]:

domain_str = """



fluent(occupied(P)) :- loc(P).
fluent(at(R,P)) :- robot(R), loc(P).
action(move(R,P)) :- robot(R), loc(P).

causes(move(R,P2), at(R,P2), at(R,P1)) :- robot(R), adjacent(P1, P2).
causes(move(R,P), at(R,P), at(R,P)) :- robot(R).
if(occupied(P), at(_, P)).
impossible(move(R,P), occupied(P)) :- robot(R), loc(P).
impossible(move(R,P2), at(R,P1)) :- robot(R), loc(P1), loc(P2), not adjacent(P1, P2).

"""

In [53]:
domain_p = from_string(domain_str)[0]
print(domain_p.custom_str(sep='\n'))

#program base.
fluent(occupied(P)) :- loc(P).
fluent(at(R,P)) :- robot(R), loc(P).
action(move(R,P)) :- robot(R), loc(P).
causes(move(R,P2),at(R,P2),at(R,P1)) :- robot(R), adjacent(P1,P2).
causes(move(R,P),at(R,P),at(R,P)) :- robot(R).
if(occupied(P),at(_,P)).
impossible(move(R,P),occupied(P)) :- robot(R), loc(P).
impossible(move(R,P2),at(R,P1)) :- robot(R), loc(P1), loc(P2), not adjacent(P1,P2).


In [54]:
__t = Function('__t')
one = Term(IntegerConstant(1))

In [55]:
action_language_rules = []
for rule in domain_p.rules:
    if rule.is_rule() and not rule.is_constraint():
        if rule.is_normal_rule():
            symbol = rule.head.atom.symbol
            if symbol.name == 'causes':
                effect = BasicLiteral(Atom(symbol.arguments[1].arguments_append(__t + one)))
                action = BasicLiteral(Atom(symbol.arguments[0].arguments_append(__t)))
                conditions = (BasicLiteral(Atom(condition.arguments_append(__t))) for condition in symbol.arguments[2:])
                al_head = effect
                al_body = (*rule.body,
                           action,
                           *conditions)
                al_rule = NormalRule(al_head, al_body)
                action_language_rules.append(al_rule)
            elif symbol.name == 'impossible':
                head = rule.head
                action = BasicLiteral(Atom(symbol.arguments[0].arguments_append(__t)))
                preconditions = (BasicLiteral(Atom(condition.arguments_append(__t))) for condition in symbol.arguments[1:])
                al_body = (*rule.body,
                           action,
                           *preconditions)
                al_rule = Constraint(al_body)
                action_language_rules.append(al_rule)
            elif symbol.name == 'if':
                head = rule.head
                side_effect = BasicLiteral(Atom(symbol.arguments[0].arguments_append(__t)))
                effects = (BasicLiteral(Atom(effect.arguments_append(__t))) for effect in symbol.arguments[1:])
                al_head = side_effect
                al_body = (*rule.body,
                           *effects)
                al_rule = NormalRule(al_head, al_body)
                action_language_rules.append(al_rule)


action_language_p = Program(name='action_language', parameters=(__t,), rules=action_language_rules)
print(action_language_p.custom_str(sep='\n'))

#program action_language(__t).
at(R,P2,__t+1) :- robot(R), adjacent(P1,P2), move(R,P2,__t), at(R,P1,__t).
at(R,P,__t+1) :- robot(R), move(R,P,__t), at(R,P,__t).
occupied(P,__t) :- at(_,P,__t).
:- robot(R), loc(P), move(R,P,__t), occupied(P,__t).
:- robot(R), loc(P1), loc(P2), not adjacent(P1,P2), move(R,P2,__t), at(R,P1,__t).


In [56]:
inertia_str = """

#program action_language(__t).

%occupied(P, __t+1) :- occupied(P, __t), not -occupied(P,__t+1).
%-occupied(P, __t+1) :- -occupied(P, __t), not occupied(P,__t+1).

%:- loc(P), not occupied(P, __t), not -occupied(P, __t).

"""
inertia_p = from_string(inertia_str)[1]
print(inertia_p.custom_str(sep='\n'))

#program action_language(__t).


In [57]:
instance_str = """

row(1). col(1).
row(2). col(2).
row(3). col(3).
loc((X,Y)) :- col(X), row(Y).

adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X,Y+1).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X+1,Y).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X,Y-1).
adjacent(P1, P2) :- loc(P1), loc(P2), P1 = (X,Y), P2 = (X-1,Y).

robot(worker).
robot(charger).

-occupied((1,2),0).
occupied((1,3),0).
-occupied((2,1),0).
-occupied((2,3),0).
occupied((3,1),0).
-occupied((3,2),0).
-occupied((3,3),0).

"""
instance_p = from_string(instance_str)[0]
print(instance_p.custom_str(sep='\n'))

#program base.
row(1).
col(1).
row(2).
col(2).
row(3).
col(3).
loc((X,Y)) :- col(X), row(Y).
adjacent(P1,P2) :- loc(P1), loc(P2), P1=(X,Y), P2=(X,Y+1).
adjacent(P1,P2) :- loc(P1), loc(P2), P1=(X,Y), P2=(X+1,Y).
adjacent(P1,P2) :- loc(P1), loc(P2), P1=(X,Y), P2=(X,Y-1).
adjacent(P1,P2) :- loc(P1), loc(P2), P1=(X,Y), P2=(X-1,Y).
robot(worker).
robot(charger).
-occupied((1,2),0).
occupied((1,3),0).
-occupied((2,1),0).
-occupied((2,3),0).
occupied((3,1),0).
-occupied((3,2),0).
-occupied((3,3),0).


In [58]:
plan_str = """

#program init.

1 { at(R,P,0) : loc(P) } 1 :- robot(R).

#program plan(__t).

1 { move(R,P,__t) : loc(P) } 1 :- robot(R).

%:- { at(R,P,__t) : loc(P) } 0, robot(R).
%:- 2 { at(R,P,__t) : loc(P) } , robot(R).

#program init.

:- not at(worker, (1,1), 0).
:- not at(charger, (2,2), 0).

#program goal(__t).

:- not at(worker, (3,3), __t).
:- not at(charger, (2,2), __t).

"""

plan_ps = from_string(plan_str)[1:]
print(*(p.custom_str(sep='\n', end='\n') for p in plan_ps),sep='\n')

#program init.
1 <= {at(R,P,0): loc(P)} <= 1 :- robot(R).

#program plan(__t).
1 <= {move(R,P,__t): loc(P)} <= 1 :- robot(R).

#program init.
:- not at(worker,(1,1),0).
:- not at(charger,(2,2),0).

#program goal(__t).
:- not at(worker,(3,3),__t).
:- not at(charger,(2,2),__t).



In [59]:
ps = (action_language_p, inertia_p, instance_p, *plan_ps)
print(*(p.custom_str(sep='\n', end='\n') for p in ps), sep='\n\n')

#program action_language(__t).
at(R,P2,__t+1) :- robot(R), adjacent(P1,P2), move(R,P2,__t), at(R,P1,__t).
at(R,P,__t+1) :- robot(R), move(R,P,__t), at(R,P,__t).
occupied(P,__t) :- at(_,P,__t).
:- robot(R), loc(P), move(R,P,__t), occupied(P,__t).
:- robot(R), loc(P1), loc(P2), not adjacent(P1,P2), move(R,P2,__t), at(R,P1,__t).


#program action_language(__t).


#program base.
row(1).
col(1).
row(2).
col(2).
row(3).
col(3).
loc((X,Y)) :- col(X), row(Y).
adjacent(P1,P2) :- loc(P1), loc(P2), P1=(X,Y), P2=(X,Y+1).
adjacent(P1,P2) :- loc(P1), loc(P2), P1=(X,Y), P2=(X+1,Y).
adjacent(P1,P2) :- loc(P1), loc(P2), P1=(X,Y), P2=(X,Y-1).
adjacent(P1,P2) :- loc(P1), loc(P2), P1=(X,Y), P2=(X-1,Y).
robot(worker).
robot(charger).
-occupied((1,2),0).
occupied((1,3),0).
-occupied((2,1),0).
-occupied((2,3),0).
occupied((3,1),0).
-occupied((3,2),0).
-occupied((3,3),0).


#program init.
1 <= {at(R,P,0): loc(P)} <= 1 :- robot(R).


#program plan(__t).
1 <= {move(R,P,__t): loc(P)} <= 1 :- robot(R).


#program

In [60]:
list(evaluate_forwards((*ps, "#show at/3. #show move/3. #show occupied/2. #show -occupied/2."), parts=(
    ('base',()),
    ('init',()),
    ('goal',(clingo.Number(4),)),
    *(('plan', (clingo.Number(n),)) for n in range(0,4)),
    *(('action_language', (clingo.Number(n),)) for n in range(0,5)),
), report=True));

Answer 1: {
occupied((1,1),0)
occupied((1,3),0)
occupied((2,1),1)
occupied((2,2),0)
occupied((2,2),4)
occupied((3,1),0)
occupied((3,1),2)
occupied((3,2),1)
occupied((3,2),3)
occupied((3,3),4)
at(charger,(2,2),0)
at(charger,(2,2),4)
at(charger,(3,1),2)
at(charger,(3,2),1)
at(charger,(3,2),3)
at(worker,(1,1),0)
at(worker,(2,1),1)
at(worker,(3,1),2)
at(worker,(3,2),3)
at(worker,(3,3),4)
move(charger,(2,2),3)
move(charger,(3,1),1)
move(charger,(3,2),0)
move(charger,(3,2),2)
move(worker,(2,1),0)
move(worker,(3,1),1)
move(worker,(3,2),2)
move(worker,(3,3),3)
-occupied((1,2),0)
-occupied((2,1),0)
-occupied((2,3),0)
-occupied((3,2),0)
-occupied((3,3),0)
}
Answer 2: {
occupied((1,1),0)
occupied((1,3),0)
occupied((2,1),1)
occupied((2,2),0)
occupied((2,2),4)
occupied((3,1),0)
occupied((3,1),2)
occupied((3,2),1)
occupied((3,2),3)
occupied((3,3),2)
occupied((3,3),4)
at(charger,(2,2),0)
at(charger,(2,2),4)
at(charger,(3,2),1)
at(charger,(3,2),3)
at(charger,(3,3),2)
at(worker,(1,1),0)
at(worker,(2,1)